In [ ]:
# System imports
import os
from os import path
import math
import corner_detection_utils
import piece_detection_utils

# Third-Party Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from sklearn.cluster import DBSCAN as db



In [ ]:
#Test to see if we can load in an image (May remove this later)
datadir = 'Test_Images/'
image_file = 'Othello_Game_Board5.png'
full_path = os.path.join(datadir, image_file)
img = plt.imread(full_path)
plt.imshow(img)

In [ ]:
datadir = 'Test_Images/'
def detect_corners(image_output_dir, corner_output_dir, corner_detection_func):
    image_output_dir = image_output_dir
    corner_output_dir = corner_output_dir
    os.makedirs(image_output_dir, exist_ok=True)
    os.makedirs(corner_output_dir, exist_ok=True)
    for i in range(1,11): #Switch to 11 later
        # Choose the image file
        image_file = f'Othello_Game_Board{i}.png'
        # Detect corners
        try: 
            # Read the image
            img = np.float32(cv2.imread(datadir+image_file, cv2.IMREAD_COLOR_RGB) / 255.0)
            corners = corner_detection_func(img)

            # Plot the image and corners
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(img)

            #Check if corner is numpy array if so convert to lsit
            if isinstance(corners, np.ndarray):
                corners_list = corners.tolist()
            else:
                corners_list = corners

            for corner in corners:
                ax.scatter(corner[0], corner[1], color='yellow', s=40, marker='o')

            # Save the image
            image_output_file = os.path.join(image_output_dir, image_file)
            plt.savefig(image_output_file, bbox_inches='tight', pad_inches=0.1)
            plt.show()
            plt.close(fig)  # Close the figure to free up memory

            # Save the corners to a .txt file
            corner_output_file = os.path.join(corner_output_dir, image_file.replace('.png', '.txt'))
            with open(corner_output_file, 'w') as f:
                f.write(str(corners_list))

            print(f"Image saved to: {image_output_file}")
            print(f"Corners saved to: {corner_output_file}")
        except Exception as e:
            print(f"Error processing {image_file} {e}")
            continue

In [ ]:
def idenitfy_board_pieces(corner_dir):
    # Set directories
    datadir = 'Test_Images/'
    # Iterate over all 10 Othello board images
    for i in range(1, 11):
        # Construct image and corner file names
        image_file = f'Othello_Game_Board{i}.png'
        corner_file = f'Othello_Game_Board{i}.txt'
        full_image_path = os.path.join(datadir, image_file)
        full_corner_path = os.path.join(corner_dir, corner_file)

        # Check if both the image and corner file exist
        if not os.path.exists(full_image_path):
            print(f"Image not found, skipping: {image_file}")
            continue

        if not os.path.exists(full_corner_path):
            print(f"Corner file not found, skipping: {corner_file}")
            continue

        # Load the image
        input_image = np.float32(cv2.imread(full_image_path, cv2.IMREAD_COLOR) / 255.0)
        plt.imshow(input_image)
        plt.title(image_file)
        plt.show()

        # Load the corners
        with open(full_corner_path, 'r') as f:
            corners = eval(f.read())

        # Convert corners to a NumPy array
        corners = np.array(corners, dtype=np.float32)
        print(len(corners))

        # Skip we don't have 4 corners
        if len(corners) != 4: #Note we need exactly for corners for piece detection algorithm to work 
            print("There aren't exactly 4 corners that were detected")
            continue

        # Extract the board state
        board_state = np.zeros((8, 8))
        for row in range(8):
            for col in range(8):
                piece = piece_detection_utils.detect_piece(input_image, corners, row, col)
                board_state[row][col] = piece[0]

        # Print the final board state
        print(f"Board State for {image_file}:")
        print(board_state)


In [ ]:
#1. Detect Corners using Hough Algorithm (Store the resultant images of corners being drawn as well as the individual coordinates)
image_output_dir = 'Corners_Extracted_Hough_Images/'
corner_output_dir = 'Corners_Extracted_Hough/'
detect_corners(image_output_dir, corner_output_dir, corner_detection_utils.hough)

#2. Now identify the pieces in each original image with the help of the corners
idenitfy_board_pieces(corner_output_dir)

In [ ]:
#1. Detect Corners using Chroma Algorithm (Store the resultant images of corners being drawn as well as the individual coordinates)
image_output_dir = 'Corners_Extracted_Chroma_Images/'
corner_output_dir = 'Corners_Extracted_Chroma/'
detect_corners(image_output_dir, corner_output_dir, corner_detection_utils.chroma_key)

#2. Now identify the pieces in each original image with the help of the corners
idenitfy_board_pieces(corner_output_dir)

In [ ]:
#1. Detect Corners using Fast Line Detector Algorithm (Store the resultant images of corners being drawn as well as the individual coordinates)
image_output_dir = 'Corners_Extracted_FastLineDetector_Images/'
corner_output_dir = 'Corners_Extracted_FastLineDetector/'
detect_corners(image_output_dir, corner_output_dir, corner_detection_utils.fast_line_detector)

#2. Now identify the pieces in each original image with the help of the corners
idenitfy_board_pieces(corner_output_dir)